# Ethnocentrism in Romeo and Juliet
### Created by Joshua Mateer

Welcome to a case study where we study how ethnocentrism plays a role in Romeo and Juliet. This is still very much a work in progress, but I hope you enjoy it! Let's start by importing the necessary libraries.

In [1]:
# Import necessary libraries
import random
import math

Here are some paramters that the program will use to generate the dialogue. You can edit everything in the cell below and the "story" will adjust accordingly. 

In [2]:
# tells the program which attribute to look at for conflicts
# Feel free to edit the following values to see how the program adjusts!
# 0 = Make decisions based on family
# 1 = Make decisions based on gender
# 2 = Make decisions based on age
# 3 = Make decisions based on wealth
currentGoal = 0

# probablities for each class
OUT_HELP_ETHNO = 0.2 # How likely is a ethnocentric agent to help someone outside their group?
IN_HELP_ETHNO = 0.975 # How likely is a ethnocentric agent to help someone within their group?
OUT_HELP_COSMO = 0.95 # How likely is a cosmopolitan agent to help someone outside their group?
IN_HELP_COSMO = 0.3 # How likely is a cosmopolitan agent to help someone within their group?
OUT_HELP_EGO = 0.1 # How likely is a egotistic agent to help someone outside their group?
IN_HELP_EGO = 0.1 # How likely is a egotistic agent to help someone within their group?
OUT_HELP_ALTRU = 0.95 # How likely is a altruistic agent to help someone outside their group?
IN_HELP_ALTRU = 0.925 # How likely is a altruistic agent to help someone within their group?

Don't mind this. I'm just turning all the data you just gave me into a list that the program can read. Reduces code later and allows the program to scale if I want to add/change something later.

In [3]:
# Don't edit this. This is how the program creates the variable conflict system
OPD = [[OUT_HELP_ETHNO, IN_HELP_ETHNO],[OUT_HELP_COSMO, IN_HELP_COSMO],[OUT_HELP_EGO, IN_HELP_EGO],[OUT_HELP_ALTRU, IN_HELP_ALTRU]]

Let's create our model! This model features four custom objects: a Dialogue class that manages individual lines of dialogue, a DialogueManager class that will manage all dialogue and intentions for a character, an Agent class that stores data for our indivdual characters (including their DialogueManager), and a ModelManager class that will dictate how characters interact and keep track of their interactions.

In [8]:
# A new object that will help in dialogue generation
class Dialogue:
    speaker = ""
    text = ""
    addressee = ""

    def __init__(self, speaker, text, addressee):
        self.speaker = speaker
        self.text = text
        self.addressee = addressee

# A new object that will manage each character's dialogue
class DialogueManager:
    library = []

    def __init__(self):
        self.library = []
    
    # add a piece of dialogue to this agent's library
    def add(self, dialogue):
        self.library.append(dialogue)
    
    # return the size of this agent's library
    def size(self):
        return len(self.library)
    
    def get(self, n):
        return self.library[n]
    
    def findInteraction(self, target):
        choices = []
        for d in self.library:
            if d.addressee == target.name:
                choices.append(d)
        if len(choices) > 0:
            return random.choice(choices).text
        return random.choice(self.library).text
    
# A new object that will dictate how the characters interact
class Agent:
    name = ""
    types = []
    moTypes = []
    dialogue = 0 # null in python, will be turned into a DialogueManager later

    # initalization
    def __init__(self, name, types, moTypes):
        self.name = name
        self.types = types
        self.moTypes = moTypes
        self.dialogue = DialogueManager()

    # add a piece of dialogue to this character's library
    def addDialogue(self, text, target):
        self.dialogue.add(Dialogue(self.name, text, target))

    # create an converstaion between this character and a target
    def speakWith(self, target):
        print(self.name+": "+self.dialogue.findInteraction(target))
        
    # decide whether this agent will help another agent
    def decideToHelp(self, target):
        threshold = 0
        if self.types[currentGoal] == target.types[currentGoal]:
            threshold = OPD[self.moTypes[currentGoal]][1]
        else:
            OPD[self.moTypes[currentGoal]][0]
        return (decision := random.uniform(0,1)) < threshold
    
# This is the object that will make the magic happen. Hooray.
class ModelManager:
    agents = []
    
    def __init__(self):
        self.agents = []

    # add an agent to the model
    def add(self, agent):
        self.agents.append(agent)
    
    # return the size of the agents array
    def size(self):
        return len(self.agents)
    
    # create an interaction between two agents
    def createInteraction(self, agent1, agent2):
        agent1.speakWith(agent2)
        agent2.speakWith(agent1)
        if agent1.decideToHelp(agent2) and agent2.decideToHelp(agent1):
            print("Hooray! "+agent1.name+" and "+agent2.name+" had a positive interaction!")
        else:
            print("Oh no! "+agent1.name+" and "+agent2.name+" had a negative interaction.")
        print("\n")
    
    # run the simulation one time
    def runSim(self):
        random.shuffle(self.agents)
        for i in range(0, math.floor(len(self.agents)/2)):
            # create an interaction between each pair of characters in the story.
            self.createInteraction((pair0 := self.agents[i*2]), (pair1 := self.agents[i*2+1]))

At this point, we're finally ready to make some characters! Here are some sample characters I made. Each character has a set of attributes that the program can access to check interactions. Additionally, each character has a set of numbers that represents how they will interact with other characters in a given conflict situation.  
Key:
- a 0 represents that they are ethnocentric with respect to this conflict
- a 1 represents that they are cosmopolitan with respect to this conflict
- a 2 represents that they are egotistic with respect to this conflict
- a 3 represents that they are altruistic with respect to this conflict

- The first number represents their response to family conflict
- The second number represents their response to gender conflict
- The third number represents their response to age conflict
- The fourth number represents their response to wealth conflict

In [9]:
mod = ModelManager()

# Let's create all the different characters and set how they'll interact
# Feel free to edit the numbers at the end to change how they interact with different groups of people
mod.add(Agent("MERCUTIO", ["Montague", "male", "young", "rich"], [0, 3, 2, 3]))
mod.add(Agent("MONTAGUE", ["Montague", "male", "old", "rich"], [2, 2, 2, 3]))
mod.add(Agent("ROMEO", ["Montague", "male", "young", "rich"], [3, 3, 0, 3]))
mod.add(Agent("BENVOLIO", ["Montague", "male", "young", "rich"], [0, 3, 2, 3]))
mod.add(Agent("ABRAM", ["Montague", "male", "young", "rich"], [2, 3, 2, 3]))
mod.add(Agent("BALTHASAR", ["Montague", "male", "young", "rich"], [0, 3, 2, 3]))
mod.add(Agent("CAPULET", ["Capulet", "male", "old", "rich"], [2, 2, 2, 3]))
mod.add(Agent("JULIET", ["Capulet", "female", "young", "rich"], [3, 3, 0, 3]))
mod.add(Agent("TYBALT", ["Capulet", "male", "young", "rich"], [0, 2, 2, 3]))
mod.add(Agent("NURSE", ["Capulet", "female", "old", "rich"], [3, 3, 1, 3]))
mod.add(Agent("PETER", ["Capulet", "male", "young", "rich"], [3, 3, 3, 3]))
mod.add(Agent("SAMPSON", ["Capulet", "male", "young", "rich"], [2, 2, 2, 3]))
mod.add(Agent("GREGORY", ["Capulet", "male", "young", "rich"], [2, 2, 2, 3]))
mod.add(Agent("FRIAR LAWRENCE", ["Neither", "male", "old", "rich"], [3, 3, 3, 3]))
mod.add(Agent("PRINCE", ["Neither", "male", "old", "rich"], [0, 3, 3, 3]))

Finally, we're ready to actually read Romeo and Juliet! We will read in the file, walk through our character list and find all of their dialogoue. Eventually, I'd like to figure out how to have the model predict who each line is addressed to, but right now that doesn't work.

In [10]:
# Let's get some dialogue set up.

# Open the text file 
play = open('romeo_juliet.txt')

# Read the text from the file
play_text = play.read()

# Iterate through all characters in the model:
for agent in mod.agents:
    start = 0
    # Inner loop will iterate until it can no longer find a
    # character's name in the body of text
    while start != -1: 
        # Find the first occurance of a character's name
        start = play_text.find(agent.name + '.\n') 
        
        # Truncate the text so that it starts from the location
        # of the character's name. Store truncated text in a 
        # temp variable
        temp = play_text[start:]

        # Find where a character's statement ends.  If you examine the text
        # you will find that in each dialog characters' statements are separated
        # by two blank lines 
        stop = temp.find('\n\n')
        
        # Extract an individual statement by slicing the text
        # stored in the temp variable from the beginning of the text
        # to the end of the statement (location of a double blank line)
        statement = temp[:stop]
        
        # Remove statement from the body of text
        play_text = play_text.replace(statement, '')
        statement = statement.replace(agent.name + '.\n', '')
        
        # Try to figure out who speaks next after the line and add the dialogue to the character.
        speaksNext = play_text[:play_text.find(".\n")]
        speaksNext = speaksNext.replace(".\n", '')
        
        # Don't add any blank dialogue for goodness sakes.
        if statement != "":
            agent.addDialogue(statement, speaksNext)

## The Simulation

Now we're finally ready to run the sim! Just press run on the cell below to watch the story unfold before your very eyes. 

In [11]:
# Just press run on this cell to see an entirely new story unfold!
mod.runSim()

JULIET: My ears have yet not drunk a hundred words
Of thy tongue's utterance, yet I know the sound.
Art thou not Romeo, and a Montague?
PETER: I saw no man use you at his pleasure; if I had, my weapon should
quickly have been out. I warrant you, I dare draw as soon as another
man, if I see occasion in a good quarrel, and the law on my side.
Hooray! JULIET and PETER had a positive interaction!


MONTAGUE: Many a morning hath he there been seen,
With tears augmenting the fresh morning's dew,
Adding to clouds more clouds with his deep sighs;
But all so soon as the all-cheering sun
Should in the farthest east begin to draw
The shady curtains from Aurora's bed,
Away from light steals home my heavy son,
And private in his chamber pens himself,
Shuts up his windows, locks fair daylight out
And makes himself an artificial night.
Black and portentous must this humour prove,
Unless good counsel may the cause remove.
TYBALT: Mercutio, thou consortest with Romeo.
Oh no! MONTAGUE and TYBALT had a n